# Venue Similarity between Neighborhoods in New York, Toronto and Detroit #

## 1. Introduction/Business Problem ##

In this project, I'll be using venue data from neighborhoods gathered from Foursquare to determine the similarity or dissimilarity between New York, Toronto and Detroit. The targeted audience in this situation would be residents of said cities who are looking to move to a different city in the list. The audience will be able to select the city that is most similar to their own in terms of types of venues available.

## 2. Data ##

The data that will be used to solve this problem is a combination of neighborhood data gathered by scraping public sites, as well as geographic and venue data gathered from Foursquare. For example, I'll be gathering data about Detroit's neighborhoods using the following Wikipedia article: https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Detroit, including neighborhood, borough, latitude and longitude. I'll be gathering venue data using Foursquare's Venues API: https://developer.foursquare.com/docs/api-reference/venues/search/. For each neighborhood, I'll use the Venues API to gather the top 10 venue types.

## 3. Methodology ##

### 3.1 Methodology overview ###

To begin this process, I needed to ensure that I had dataframes for New York, Toronto, and Detroit that included all cities, boroughs, neighborhoods, and their latitudes and longitudes. Latitudes and longitudes were crucial datapoints to capture as they were required parameters in the REST API calls made to the Foursquare Venues Explore API. Calls to this API return venues in a defined range of each latitude/longitude pair, along with the category of the venue. I was able to use the category field to create dataframes containing all neighborhoods and their frequency of each venue category. I was then able to combine the venue and city data for all three cities. Using kmeans clustering, I created several clusters based on the most frequent venue categories. From this, I was able to determine which two cities were clustered together the most. This gave me my final result of the two cities out of New York, Toronto, and Detroit that were the most similar in terms of the most common venue categories in all of their neighborhoods. This data can be used to help residents of said cities decide which city to move to based on the similarity of their venue categories.

### 3.1 Gathering data ###

Data for each city was available in separate locations on the public internet. For this reason, I started by creating separate dataframes for each city. This section describes the process of gathering this data.

#### 3.1.1 New York ####

I was able to gather this data for New York using a JSON file that was publicly available in AWS S3 cloud storage at the following address:
https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json

Once the JSON file was retrieved, I was able to create a dataframe by iterating through objects in the JSON file and gathering their city, boroughs, neighborhoods, latitudes, and longitudes. 

#### 3.1.2 Toronto ####

I was able to gather data for Toronto by scraping the folllowing Wikipedia site using Python's BeautifulSoup library:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

The Wikipedia site contained tables with all neighborhoods, along with their postal codes and boroughs. Postal codes were not included in data retrieved for New York or Detroit, so this data was ignored. A dataframe was created from the data retrieved by scraping this site. This site did not contain latitude and longitude data. I was able to retrieve latitude and longitude data using a CSV file available in AWS S3 cloud storage at the following address:
https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv

Once latitude and longitude data was retrieved, I was able to create a dataframe from it and merge it with the dataframe created by scraping the Wikipedia site. Only then did I have a dataframe for Toronto containing city, boroughs, neighborhoods, latitudes and longitudes.

#### 3.1.3 Detroit ####

I was able to gather data for Detroit by scraping the following Wikipedia site using Python's BeautifulSoup library: https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Detroit

The Wikipedia site contained several tables with neighborhoods. Each table contained neighborhoods for a separate borough. Some entries in these tables contained latitude and longitude data and some didn't. Since I didn't have a separate data source containing latitude and longitude data for each neighborhood, I decided to only capture neighborhoods whose latitude and longitude were listed.

Once I scraped the Wikipedia site for the proper data, I was able to easily convert it to a dataframe containing city, boroughs, neighborhoods, latitudes and longitudes.

### 3.2 Foursquare API ###

After creating dataframes for each city containing city, borough, neighborhood, latitude and longitude I was ready to call the Foursquare Venues Explore API to retrieve venues and venue categories for each city. I made use of a function that takes neighborhood names, latitude, longitude, and range as arguments. This function iterates through each neighborhood and executes an API call that returns the venues and venue categories in the range of the latitude and longiudes for each neighborhood as a dataframe. This function was executed for each city.

### 3.3 Data manipulation and combination ###

Once I had a dataframe for each city containing every neighborhood and their nearby venues, I needed to manipulate the data to include frequencies of each venue type for each neighborhood. To do this, I created new dataframes containing neighborhoods and the quantities of every venue category. From this I was able to create another set of dataframes, where each value contained one neighborhood, and mean values of venue categories. Once I had these dataframes for each city, I was able to concatenate them together to form a single dataframe with mean values of venue categories. Since not every city contained all venue types, I dropped columns that contained no data for some cities. After this some rows still contained no data, so these were dropped as well. This eliminated the possibility of error using machine learning algorithms.

### 3.4 Machine Learning ###

To determine similarity between neighborhoods, I opted to use kmeans clustering. The overall objective of the kmeans clustering algorithm is to group data points together based on similarity for the purposes of discovering patterns. Kmeans clustering would allow me to create clusters of neighborhoods that had simliar top venue categories. Since there was such as large dataset to work with, I decided to use 10 kclusters for kmeans clustering. This gave me enough clusters to determine the pattern of which two cities were the most related. When combined with previous dataframes that contained the city for each neighborhood, I was able to look at the cities of each cluster that had similar venue categories. This allowed me to capture the cities that had the most similarities in terms of venue categories.

## 4. Results ##

After running kmeans clustering and searching through the clusters for the cities with similar venue categories, it was determined that New York and Toronto were the most similar cities in terms of their most common venue categories. New York and Toronto were the two most common cities in 5 of the 10 clusters, followed by New York and Detroit with 2 of the 10 clusters. Toronto and Detroit were never the most common cities in any cluster, meaning they are the least similar in terms of venue categories. 3 clusters were unused to determine the final results either because they contained neighborhoods from only 1 city, or because the number of neighborhoods from each city was equal.

## 5. Discussion ##

Based on these results, I can confidently make the recommendation that anyone from New York looking to move to Toronto or Detroit who favors the popular venue categories of New York should choose Toronto. The same goes for anyone looking to move from Toronto to either New York or Detroit - New York should be chosen based on similar popular venue categories. Since New York and Detroit were the second most similar cities, anyone from Detroit looking to move to New York or Toronto should choose New York based on similar popular venue categories.

The biggest observation that was made during this project was the difference in sizes of the city dataframes. New York's data contained 300 neighborhoods, with Toronto's containing 102 neighborhoods and Detroit containing only 48 neighborhoods. It is possible that the difference in number of neighborhoods influenced the final results, as it is statistically more likely that clusters would contain more neighborhoods from New York than Toronto or Detroit. I noted that Detroit had the fewest number of neighborhoods likely because not all neighborhoods on the Wikipedia page that was scraped for data contained latitude and longitude data. For this reason, only neighborhoods that contained latitude and longitude data were used.

The recommendation that I would give to improve this project in the future would be to ensure that the sizes of all city dataframes are relatively similar to eliminate statistical biases towards a single city. In this case, initial research on common New York boroughs could have allowed me to have limited the New York neighborhoods to those that are more populous.

## 6. Conclusion ##

The process of determining the most similar cities in terms of common venue categories was made possible due to a combination of publicly available CSV and JSON files, public web scraping, Foursquare's Venues Explore API, data cleaning, manipulation and combination, and kmeans clustering. Improvements to this process can be made, and have been outlined in the Discussion section. I would still recommend that any future data scientists use simlar methods with web scraping, the Foursquare Venues Explore API and kmeans clustering.

In the end it was determined that New York and Toronto are the most similar cities in terms of popular venue categories, with New York and Detroit being the second most simliar. Recommendations to potential movers to/from these cities can be made based on this data.